In [1]:
import os
from pathlib import Path
import random

import tensorflow as tf
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

from model_def import Generator, Discriminator

2024-05-13 21:05:28.334834: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 21:05:29.165640: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def jpg_to_jpeg(dir_path):
    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)

        if file_path.endswith('.jpg') or file_path.endswith('.JPG'):
            img = cv2.imread(file_path)
            cv2.imwrite(file_path[:-4] + ".jpeg", img)

            os.remove(file_path)

In [3]:
DATASET_PATH = "../training_dataset"

# Tranform images from jpg format to jpeg format
jpg_to_jpeg(DATASET_PATH)

# save the photo in the form numpy array


training_set = []

for img_name in os.listdir(TRAIN_DATASET_PATH):
    img_path = os.path.join(TRAIN_DATASET_PATH, img_name)
    
    training_set.append(cv2.imread(img_path))

training_set = tf.convert_to_tensor(training_set)

print(type(training_set), training_set.shape)

np.save(os.path.join(TRAIN_DATASET_PATH, 'photo_tensor.npy'), training_set.numpy())


In [4]:
TRAIN_DATASET_PATH = os.path.join(DATASET_PATH, "photo_jpg", "photo_tensor.npy")
MONET_DATASET_PATH = os.path.join(DATASET_PATH, "monet_jpg", "monet_tensor.npy")

In [5]:
# photo_set is for the picture which we would like to convert it to Monet
photo_set = tf.convert_to_tensor(np.load(TRAIN_DATASET_PATH), dtype=tf.float32)

photo_set_training, photo_set_testing = photo_set[:5000], photo_set[5000:]

# monet_set is for the Monet
monet_set = tf.convert_to_tensor(np.load(MONET_DATASET_PATH), dtype=tf.float32)

2024-05-13 21:05:32.992604: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-13 21:05:33.037704: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-13 21:05:33.037866: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

# PLAN

### Train generator through PHOTO_SET  
### Train discriminator through both the result of generator and some images in MONET_SET 

In [13]:
CONFIG = dict({
    "EPOCHS": 10,
    "BATCH_SIZE": 16,
})

In [14]:
generator = Generator(
    origin_output_shape=(32, 32, 3),
    num_of_conv=5,
    filters=[32, 64, 128, 64, 3],
    kernel_size=[(3, 3), (3, 3), (3, 3), (3, 3), (3, 3)],
    strides=[1, 1, 1, 1, 1],
)

discriminator = Discriminator(
    num_of_dense=3,
    dense_units=[64, 64, 32],
    num_of_conv=4,
    filters=[64, 128, 128, 64],
    kernel_size=[(3, 3), (3, 3), (3, 3), (3, 3)],
    strides=[2, 2, 2, 1]
)

In [15]:
discriminator.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=["accuracy"]
)

In [16]:
discriminator.trainable = False

model_input = tf.keras.Input(shape=(256, 256, 3), name="model_input")
model_output = discriminator(generator(model_input))

model = tf.keras.models.Model(model_input, model_output)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [17]:
def train_discriminator(real_images, fake_images):
    real = np.ones(shape=(CONFIG["BATCH_SIZE"], 1), dtype=np.float32)
    fake = np.zeros(shape=(CONFIG["BATCH_SIZE"], 1), dtype=np.float32)

    # Train on real images
    real_history = discriminator.train_on_batch(x=real_images, y=real)

    # Train on fake images
    fake_history = discriminator.train_on_batch(x=fake_images, y=fake)

    return real_history, fake_history


def train_generator(images):
    return model.train_on_batch(x=images, y=np.ones(shape=(CONFIG["BATCH_SIZE"], 1), dtype=np.float32))

In [18]:
%%time

# we do not need to train the discriminator with mixture of fake and real monet
# train it seperately instead.

discriminator_history = []
generator_history = []

for _ in range(CONFIG["EPOCHS"]):
    monet_idx = random.sample(range(monet_set.shape[0]), k=CONFIG["BATCH_SIZE"])
    monet_batch = tf.gather(monet_set, indices=monet_idx)

    photo_idx = random.sample(range(photo_set_training.shape[0]), k=CONFIG["BATCH_SIZE"])
    photo_batch = tf.gather(photo_set_training, indices=photo_idx)

    discriminator_history.append(train_discriminator(real_images=monet_batch, fake_images=generator(photo_batch)))
    generator_history.append(train_generator(images=photo_batch))


CPU times: user 5.47 s, sys: 204 ms, total: 5.67 s
Wall time: 6.31 s


In [19]:
model.evaluate(x=photo_set_testing, y=np.zeros(shape=(photo_set_testing.shape[0], 1)), batch_size=CONFIG["BATCH_SIZE"])

128/128 [==============================] - 10s 77ms/step - loss: 0.5827 - accuracy: 0.8832


[0.5826960206031799, 0.8832188248634338]